In [ ]:
import urllib3
import zipfile

http = urllib3.PoolManager()
req = http.request("GET", "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip", preload_content=False)

with open("movie.zip", 'wb') as out:
  while True:
    data = req.read(4096)
    if not data:
      break
    out.write(data)
req.release_conn()

zFile = zipfile.ZipFile("movie.zip", "r")
for fileM in zFile.namelist():
  zFile.extract(fileM)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
!ls ml-latest-small/

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


### Step 1: Implement Apriori Algorithm
In this section, we implement the Apriori algorithm with pruning

In [ ]:
import pandas as pd
from collections import defaultdict


def get_support(itemset, freq_itemset_frequencies):
  return frequencies_of_freq_itemsets[itemset]

def get_candidate_itemsets(itemsets, length):
  candidates = set()
  for itemset in itemsets:
    for other_itemset in itemsets:
      if(len(itemset.union(other_itemset)) == length):
        candidates.add(itemset.union(other_itemset))

  return candidates

def get_freq_itemsets(itemsets, txs_list,  minsup, freq_itemset_frequencies):

  freq_itemset_set = set()
  for itemset in itemsets:
    #print(itemset)
    sup = 0
    for tx in txs_list:
      if(itemset.issubset(tx) == True):
        sup +=1
    #sup =  get_support(itemset, txs_list, minsup)
    if(sup >= minsup):
      freq_itemset_set.add(itemset)
      freq_itemset_frequencies[itemset] = sup # for confidence calculation

  return freq_itemset_set


In [ ]:

# read user ratings
allRatings = pd.read_csv("ml-latest-small/ratings.csv")

#keep reviews with ratings >=3 only
ratings_new = allRatings[allRatings['rating']>=3]

#initial itemset is the set of singleton movie ids
ratings_new = ratings_new.groupby('userId')
initial_candidate_list = set() #set having individual movies as sets
txs_list = [] #database of transactions
for userId, data in ratings_new:
  #each transaction(movies reviewed by a given user) is a frozen set
  txs_list.append(frozenset(data['movieId']))
  for movie in data['movieId']:
    # initial_candidate_list is the set of all unique movies reviewed by the users
    initial_candidate_list.add(frozenset([movie]))

print("Size of candidate itemsets for k = 1: {}".format(len(initial_candidate_list)))

#keep track of the support of all the frequent itemsets which form the keys of this dictionary.
#this will be used during the calculation of the confidence values
frequencies_of_freq_itemsets = defaultdict(int)

minsup_val = 150
singleton_freq_itemset = get_freq_itemsets(initial_candidate_list, txs_list, minsup_val, frequencies_of_freq_itemsets)
print("Size of filtered itemsets for k = 1: {}".format(len(singleton_freq_itemset)))

final_itemsets = list() #list of sets where each set contains itemsets indexed by the length of itemsets
final_itemsets.append(set()) # for k=0 , dummy null set
final_itemsets.append(singleton_freq_itemset)

k = 2
prev = singleton_freq_itemset

while(len(prev) > 1):
  #create new extended itemsets
  candidates = get_candidate_itemsets(prev, k)
  prev = candidates
  print("Size of candidate itemsets for k = {}: {}".format(k, len(candidates)))
  if(len(candidates) > 0):
    result = get_freq_itemsets(candidates, txs_list, minsup_val, frequencies_of_freq_itemsets)
    if(len(result)>0):
      final_itemsets.append(result)
    prev = result
    print("Size of filtered itemsets for k = {}: {}".format(k, len(result)))
  k=k+1


Size of candidate itemsets for k = 1: 8452
Size of filtered itemsets for k = 1: 37
Size of candidate itemsets for k = 2: 666
Size of filtered itemsets for k = 2: 30
Size of candidate itemsets for k = 3: 87
Size of filtered itemsets for k = 3: 2
Size of candidate itemsets for k = 4: 1
Size of filtered itemsets for k = 4: 0


### Step 2: Print Your Association Rules

Next lets print the final association rules in the following format:

**movie_name_1, movie_name_2, ... -->
movie_name_k**

where the movie names can be fetched by joining the movieId with the file `movies.csv`. For example, one rule that you might find is:

**Matrix, The (1999),  Star Wars: Episode V - The Empire Strikes Back (1980),  Star Wars: Episode IV - A New Hope (1977),  ->
Star Wars: Episode VI - Return of the Jedi (1983)**

In [ ]:
# your code here
import itertools

minconf = 0.8

movie_info = pd.read_csv("ml-latest-small/movies.csv")

def genAssociateRules(itemset, minconf, frequencies):
  '''
  returns the association rules for a given itemset given
  the itemset, min confidence value and the frequencies(support) of all itemsets
  '''
  rules =[]
  rulecount = 0

  for length in range(1,len(itemset)+1):
    subsets_list = list(itertools.combinations(itemset, length))
    for subset in subsets_list:
      #association rule : antecedent => consequent
      antecedent = frozenset(itemset.difference(subset))
      if(len(antecedent)>0 and len(antecedent)<len(itemset)):
        num = frequencies[itemset]
        den = frequencies[antecedent]
        conf = float(num)/den
        if(conf >= minconf):
          rulecount+=1
          #get the corresponding movie names from the ids
          antecedent_movies = []
          consequent_movies = []
          for id in antecedent:
            antecedent_movies.append(movie_info[movie_info['movieId'] == id]['title'].values[0])
          for id in subset:
            consequent_movies.append(movie_info[movie_info['movieId'] == id]['title'].values[0])
          rules.append(tuple([antecedent_movies, consequent_movies]))

  return rules

In [ ]:
# read user ratings
allRatings = pd.read_csv("ml-latest-small/ratings.csv")

#keep reviews with ratings >=3 only
ratings_new = allRatings[allRatings['rating']>=3]

#initial itemset is the set of singleton movie ids
ratings_new = ratings_new.groupby('userId')
initial_candidate_list = set() #set having individual movies as sets
txs_list = [] #database of transactions
for userId, data in ratings_new:
  #each transaction(movies reviewed by a given user) is a frozen set
  txs_list.append(frozenset(data['movieId']))

In [ ]:
association_rules = []
for itemsets_of_len_k in final_itemsets:
  for itemset in itemsets_of_len_k:
    rules_list = genAssociateRules(itemset, minconf, frequencies_of_freq_itemsets)
    if(len(rules_list)>0):
      for rule in rules_list:
        formatted_rule = ''
        for movie in rule[0]:
          formatted_rule = formatted_rule + str(movie) + ','
        formatted_rule = formatted_rule.rstrip(',') + ' => '
        for movie in rule[1]:
          formatted_rule = formatted_rule + str(movie) + ','
        association_rules.append(formatted_rule.rstrip(','))

In [ ]:
i=1
for rule in association_rules:
  print(str(i) + '. ' + rule)
  i=i+1

1. Lord of the Rings: The Fellowship of the Ring, The (2001) => Lord of the Rings: The Return of the King, The (2003)
2. Lord of the Rings: The Return of the King, The (2003) => Lord of the Rings: The Fellowship of the Ring, The (2001)
3. Jurassic Park (1993) => Forrest Gump (1994)
4. Star Wars: Episode V - The Empire Strikes Back (1980) => Star Wars: Episode IV - A New Hope (1977)
5. Star Wars: Episode VI - Return of the Jedi (1983) => Star Wars: Episode IV - A New Hope (1977)
6. Lord of the Rings: The Fellowship of the Ring, The (2001) => Lord of the Rings: The Two Towers, The (2002)
7. Lord of the Rings: The Two Towers, The (2002) => Lord of the Rings: The Fellowship of the Ring, The (2001)
8. Star Wars: Episode VI - Return of the Jedi (1983) => Star Wars: Episode V - The Empire Strikes Back (1980)
9. Seven (a.k.a. Se7en) (1995) => Pulp Fiction (1994)
10. Lord of the Rings: The Return of the King, The (2003) => Lord of the Rings: The Two Towers, The (2002)
11. Lord of the Rings: The

### Step 3: Implement Random Sampling



In [ ]:
import random
import math
import numpy as np


def get_random_sampled_freq_itemsets(original_txs_list= None, original_freq_itemsets = None, alpha_list = None, relax_factor_list = None, minsup = 50, change_minsup = True, check_false_positives=True):
  '''
  inputs:
  original_txs_list : unsampled list of transactions, used for sampling
  original_freq_itemsets : set of frequent itemsets generated without sampling, used to check for false positives
  alpha_list : list of the random sampling factors(alpha) to be tested
  relax_factor_list : additional scaling factor for the minsup when random sampling is done
  minsup: minsup value to be considered (without sampling)
  change_minsup : flag used to enable/disable changign minsup value after random sampling
  check_false_positives: flag used to enable/disable checking for false positives

  Output:
  prints out the frequent itemsets for different lengths
  '''

  for alpha in alpha_list:
    print("alpha = {}".format(alpha))
    print("------------")
    for relax_factor in relax_factor_list:

      sample_size = math.ceil(len(original_txs_list)*alpha)
      sampled_txs = [original_txs_list[i] for i in sorted(random.sample(range(len(original_txs_list)), sample_size))]

      print("size of sampled database: {}".format(len(sampled_txs)))

      initial_candidate_list = set() #set having individual movies as sets

      for movie_set in sampled_txs:
        for movie in movie_set:
          initial_candidate_list.add(frozenset([movie]))

      #print(len(initial_candidate_list))
      frequencies_of_freq_itemsets = defaultdict(int)

      new_minsup = minsup
      print(" ")
      print("minsup = {}".format(minsup))

      if(change_minsup == True):
        new_minsup = minsup * alpha * relax_factor
        #print("------------")
        print("relax_factor = {}".format(relax_factor))
        print("minsup_sampled = {}".format(new_minsup))
        #print("------------")
        print(" ")

      print("Size of candidate itemsets for k = 1: {}".format(len(initial_candidate_list)))

      singleton_freq_itemset = get_freq_itemsets(initial_candidate_list, sampled_txs, new_minsup, frequencies_of_freq_itemsets)

      total_freq_itemset_count = len(singleton_freq_itemset)

      print("Size of filtered itemsets for k = 1: {}".format(len(singleton_freq_itemset)))
      print(" ")

      final_itemsets_sampled = list() #list of sets where each set contains itemsets indexed by the length of itemsets
      final_itemsets_sampled.append(set())
      final_itemsets_sampled.append(singleton_freq_itemset)

      k = 2

      prev = singleton_freq_itemset

      while(len(prev) > 1):
        #create new extended itemsets
        candidates = get_candidate_itemsets(prev, k)
        prev = candidates
        print("Size of candidate itemsets for k = {}: {}".format(k, len(candidates)))
        if(len(candidates) > 0):
          result = get_freq_itemsets(candidates, sampled_txs, new_minsup, frequencies_of_freq_itemsets)
          if(len(result)>0):
            final_itemsets_sampled.append(result)
          prev = result
          print("Size of filtered itemsets for k = {}: {}".format(k, len(result)))
          total_freq_itemset_count+=len(result)
        k=k+1
        print(" ")

      print("Total number of frequent itemsets found = {}".format(total_freq_itemset_count))

      if(check_false_positives == True):
        j=1
        false_positives = np.zeros(len(final_itemsets_sampled))
        for itemsets in final_itemsets_sampled[1:]:
          for itemset in itemsets:
            if(j < len(original_freq_itemsets)):#if original freq itemset list does not contain itemsets of this length, then mark all the freq itemsets as false positives
              if(itemset not in original_freq_itemsets[j]):
                false_positives[j]+=1
            else:
              false_positives[j] = len(final_itemsets_sampled[j])
          print("False positive count in itemsets of length k:{} = {}".format(j,int(false_positives[j])))
          j=j+1
        print("Total number of false positives found = {}".format(int(sum(false_positives))))

      print(" ")



In [ ]:
alpha_list = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9,1]
relax_factor_list = [1]
original_txs_list = txs_list
original_freq_itemsets = final_itemsets
minsup = 150
get_random_sampled_freq_itemsets(original_txs_list, original_freq_itemsets, alpha_list, relax_factor_list, minsup,False, False )

alpha = 0.4
------------
size of sampled database: 244
 
minsup = 150
Size of candidate itemsets for k = 1: 6421
Size of filtered itemsets for k = 1: 0
 
Total number of frequent itemsets found = 0
 
alpha = 0.5
------------
size of sampled database: 305
 
minsup = 150
Size of candidate itemsets for k = 1: 6240
Size of filtered itemsets for k = 1: 2
 
Size of candidate itemsets for k = 2: 1
Size of filtered itemsets for k = 2: 0
 
Total number of frequent itemsets found = 2
 
alpha = 0.6
------------
size of sampled database: 366
 
minsup = 150
Size of candidate itemsets for k = 1: 5486
Size of filtered itemsets for k = 1: 4
 
Size of candidate itemsets for k = 2: 6
Size of filtered itemsets for k = 2: 0
 
Total number of frequent itemsets found = 4
 
alpha = 0.7
------------
size of sampled database: 427
 
minsup = 150
Size of candidate itemsets for k = 1: 7448
Size of filtered itemsets for k = 1: 8
 
Size of candidate itemsets for k = 2: 28
Size of filtered itemsets for k = 2: 0
 
To

Observations:
1.   alpha is varied while keeping the minsup constant.
2.   Reducing the dataset size alone (no change in minsup) by random sampling certainly reduces the number of frequent itemsets discovered.
3.   As seen in the above results, the number of frequent itemsets increases with the increase in alpha. This is because more transactions are considered with higher value of alpha which increases the likelihood of finding a frequent itemset.



### Step 4: Check for False Positives

Next let's verify that the candidate pairs we discover by random sampling are truly frequent by comparing to the itemsets we discover over the entire dataset.

For this part, consider another parameter **minsup_sample** that relaxes the minimum support threshold. For example if we want minsup = 1/100 for whole dataset, then try minsup_sample = 1/125 for the sample. This will help catch truly frequent itemsets.


In [ ]:
alpha_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
relax_factor_list = [0.8]

original_txs_list = txs_list
original_freq_itemsets = final_itemsets

minsup_orignal = 150
get_random_sampled_freq_itemsets(original_txs_list, original_freq_itemsets, alpha_list, relax_factor_list, minsup_orignal, True, True )

alpha = 0.1
------------
size of sampled database: 61
 
minsup = 150
relax_factor = 0.8
minsup_sampled = 12.0
 
Size of candidate itemsets for k = 1: 3502
Size of filtered itemsets for k = 1: 79
 
Size of candidate itemsets for k = 2: 3081
Size of filtered itemsets for k = 2: 157
 
Size of candidate itemsets for k = 3: 1470
Size of filtered itemsets for k = 3: 57
 
Size of candidate itemsets for k = 4: 178
Size of filtered itemsets for k = 4: 8
 
Size of candidate itemsets for k = 5: 7
Size of filtered itemsets for k = 5: 0
 
Total number of frequent itemsets found = 301
False positive count in itemsets of length k:1 = 42
False positive count in itemsets of length k:2 = 131
False positive count in itemsets of length k:3 = 55
False positive count in itemsets of length k:4 = 8
Total number of false positives found = 236
 
alpha = 0.2
------------
size of sampled database: 122
 
minsup = 150
relax_factor = 0.8
minsup_sampled = 24.0
 
Size of candidate itemsets for k = 1: 4985
Size of filt

Observations:

*   In the above code, minsup is scaled proportionately in accordance with the random sampling factor (alpha)
*   The minsup is additionally scaled by a relaxation factor, set to 0.8
*   Reducing the dataset size alone (no change in minsup) by random sampling certainly reduces the number of frequent itemsets discovered as seen in the previous part.
*   Scaling the minsup along with random sampling gives more frequent itemsets.
*   Many of the frequent itemsets found this way have low frequency in the original dataset and hence do not qualify as frequent itemsets. These are marked as false positives.




### Step 5: Extensions and Next Steps

So far, we have been working with a fairly small dataset. For this last question, try your sampling-based approach on the much larger: **Movies 10M** dataset: https://files.grouplens.org/datasets/movielens/ml-10m.zip

First, we need to load this larger dataset:

In [ ]:
import urllib3
import zipfile

http = urllib3.PoolManager()
req = http.request("GET", "https://files.grouplens.org/datasets/movielens/ml-10m.zip", preload_content=False)

with open("movie.zip", 'wb') as out:
  while True:
    data = req.read(4096)
    if not data:
      break
    out.write(data)
req.release_conn()

zFile = zipfile.ZipFile("movie.zip", "r")
for fileM in zFile.namelist():
  zFile.extract(fileM)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
! ls ml-10M100K/

allbut.pl  movies.dat  ratings.dat  README.html  split_ratings.sh  tags.dat


In [ ]:
import pandas as pd
# read user ratings
allRatings = pd.read_csv("ml-10M100K/ratings.dat",sep='::', names=["userId", "movieId", "rating", "timestamp"], engine='python')
allRatings

,userId,movieId,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392
...,...,...,...,...
10000049,71567,2107,1.0,912580553
10000050,71567,2126,2.0,912649143
10000051,71567,2294,5.0,912577968
10000052,71567,2338,2.0,912578016


Now you can begin your sampling over this larger dataset.

In [ ]:
# your code here
from collections import defaultdict

#keep reviews with ratings >=3 only
ratings_new = allRatings[allRatings['rating']>=3]

#initial itemset is the set of singleton movie ids
ratings_new = ratings_new.groupby('userId')
initial_candidate_list = set() #set having individual movies as sets
txs_list = [] #database of transactions
for userId, data in ratings_new:
  txs_list.append(frozenset(data['movieId']))
  for movie in data['movieId']:
    initial_candidate_list.add(frozenset([movie]))

In [ ]:
print(len(txs_list))

69863


In [ ]:
alpha_list = [0.1]
relax_factor_list = [0.8]
original_txs_list = txs_list
original_freq_itemsets = None

minsup = 15000  # in the small movies data set, #transactions was around 600 and minsup was taken as 150. So choosing 15000 as minsup in the same proportion
get_random_sampled_freq_itemsets(original_txs_list, original_freq_itemsets, alpha_list, relax_factor_list, minsup,True, False )

alpha = 0.1
------------
size of sampled database: 6987
 
minsup = 15000
relax_factor = 0.8
minsup_sampled = 1200.0
 
Size of candidate itemsets for k = 1: 9583
Size of filtered itemsets for k = 1: 94
 
Size of candidate itemsets for k = 2: 4371
Size of filtered itemsets for k = 2: 293
 
Size of candidate itemsets for k = 3: 3324
Size of filtered itemsets for k = 3: 223
 
Size of candidate itemsets for k = 4: 871
Size of filtered itemsets for k = 4: 28
 
Size of candidate itemsets for k = 5: 60
Size of filtered itemsets for k = 5: 0
 
Total number of frequent itemsets found = 638
 



*   I have set the minsup to 15000 (proportionate to the previous case).
*   10% of the total samples are considered for analysis. Further, the minsup is scaled by 0.1 to account for the sampling and further relaxed by scaling by 0.8. This effectively makes the minsup as 1200.
*   The total number of frequent itemsets discovered are 638.

